In [ ]:
import os
import random
import sys
import numpy as np
import pandas as pd
import tensorflow as tf
SEED = 42

project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Tambahkan path tersebut ke daftar pencarian modul Python
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added '{project_root}' to Python's path")

import sys
print(sys.executable)
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

c:\Users\ASUS\AppData\Local\Programs\Python\Python312\python.exe


In [ ]:
from preprocessing.text_vectorizer import create_text_vectorizer
from util import prepare_dataset



train_df = pd.read_csv("../dataset/train.csv") 
val_df = pd.read_csv("../dataset/valid.csv")

vectorizer = create_text_vectorizer(train_df['text'].values)
label_map = {'negative': 0, 'neutral': 1, 'positive': 2}
train_df['label'] = train_df['label'].map(label_map)
val_df['label'] = val_df['label'].map(label_map)
train_ds = prepare_dataset(train_df['text'].values, train_df['label'].values, vectorizer)
val_ds = prepare_dataset(val_df['text'].values, val_df['label'].values, vectorizer)

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from sklearn.metrics import f1_score
from util import train_rnn_model


VOCAB_SIZE = 10000
SEQ_LENGTH = 100
NUM_CLASSES = train_df['label'].nunique()
rnn_units = 128
num_layers = 3
bidirectional = False
EMBEDDING_DIM = 128

trained_model, training_history = train_rnn_model(
    train_ds=train_ds,
    val_ds=val_ds,
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    rnn_units=rnn_units,
    num_classes=NUM_CLASSES,
    sequence_length=SEQ_LENGTH,
    bidirectional=bidirectional,
    num_layers=num_layers,
    epochs=20,
    patience=3,
    verbose=1
)

trained_model.save_weights(
    f"./weights/weights_layers{num_layers}_units{rnn_units}_bi{int(bidirectional)}.weights.h5"
)

print(f"Model saved to ./weights/weights_layers{num_layers}_units{rnn_units}_bi{int(bidirectional)}.weights.h5")

plt.figure()
plt.plot(training_history.history['loss'], label='train loss')
plt.plot(training_history.history['val_loss'], label='val loss')
plt.title(f"Loss - Layers: {num_layers}, Units: {rnn_units}, Bi: {bidirectional}")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
filename = f"./graph/loss_layers{num_layers}_units{rnn_units}_bi{int(bidirectional)}.png"
plt.savefig(filename)
plt.show()


y_true, y_pred = [], []
for x_batch, y_batch in val_ds:
    preds = trained_model.predict(x_batch, verbose=0)
    y_true.extend(y_batch.numpy())
    y_pred.extend(np.argmax(preds, axis=1))
f1 = f1_score(y_true, y_pred, average='macro')
print(f"F1-score: {f1:.4f} → {filename}")

with open("./f1_log.csv", "a") as log_file:
    log_file.write(f"{num_layers},{rnn_units},{int(bidirectional)},{round(f1, 4)}\n")

ModuleNotFoundError: No module named 'util'